# Snowpack

**Metadata and Data Considerations**

Data is from the National Water and Climate Center (NWCC)

Data is csv format by state. For `AZ`, `CA`, CO, NM, NV, UT, WY
Data needs to be cleaned such that it only includes data for the Colorado River Basin.
Data contains many metrics. PCA may be necessary to reduce dimensionality of the dataset.

Default format = Date | Station | Metric 1 | Metric 2 | ... | Metric n 

Data split into multiple files per state as a result of the data acquisition process. Master dataset should include all data for the Colorado River Basin.<br>
File schema as follows:

Snow_\*.csv
- Station Name
- Station ID
- Snow Water Equivalent
- Snow Depth
- Snow Density
- Precipitation Acumulation
- Snow Rain Ratio
- Air Temperature Average
- Wind Speed Average
- Elevation
- Latitude
- Longitude

Soil_\*.csv
- Station Name
- Station ID
- Soil Temperature Average

Data may differ significantly over the period of record because of the effects of climate change in the region. We will attempt to use all period of record, but failing that, we will truncate the data. Data earlier than 2010 is likely not needed for predictions in following years and is likely too enstranged from current weather regimes to be useful and may instead present more error.

Target parameter is `suf_water`, which is an engineered binary parameter that will use predicted Snow Water Equivalent (SWE), the amount of water available to the water system from snow melt, and subtract it from water usage. A `0` represents a negative resulting number (not enough water), and a `1` represents a positive resulting number (enough water). 

According to research, targeting SWE directly is more effective than targeting snow depth. We will attempt to include meteorological data as well as soil temperature measurements and elevation.

Because we are comparing with water usage and water stores, reservoir data is required to compare snowpack data to determine whether water is sufficient. Water data is taken from NWCC's RESERVOIR dataset that includes reservoir stages and storage volumes. 

Water_\*.csv
- Station Name
- Station ID
- Reservoir Storage Volume (dam^3) Start of Day Values
- Elevation
- Latitude
- Longitude

#### Notes and Caveates

Snow and water data was clipped geographically using a Colorado River Basin shapefile and ESRI ArcGIS Pro on WGS 1984 Mercator Auxiliary Sphere projection. 

CA Water and Snow data lies outside the basin boundary and will be excluded from the analysis. We will do some more research into the water resource draw CA puts on the basin system to include at the end.

Some NV snow data lies within the basin boundary. NV Water reservoirs lie outside the basin boundary. Water resource draw by NV will have to be assessed similarly to CA.

### File Organizations



### Business Problem

Colorado River Board wants to be able to predict whether or not there will be sufficient water for the year or not to determine and recommend water use regulations in Colorado River.

In [1]:
# Import general libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

In [5]:
# Importing snow files into dataframes
snow_states = ['AZ', 'CO', 'NM', 'NV', 'UT', 'WY']
raw_snow = {}

for i in snow_states:
    raw_snow[i] = pd.read_csv(f'../00_Source_Data/Snowpack/{i}/Snow_{i}_Clip.csv') 


# Importing water files into dataframes
water_states = ['AZ', 'CO', 'NM', 'UT', 'WY']
raw_water = {}

for i in water_states:
    raw_water[i] = pd.read_csv(f'../00_Source_Data/Snowpack/{i}/Water_{i}_Clip.csv') 


In [8]:
raw_snow['AZ'].head()

,Date,Station Name,Station Id,Snow Water Equivalent (mm) Start of Day Values,Snow Depth (cm) Start of Day Values,Snow Density (pct) Start of Day Values,Precipitation Accumulation (mm) Start of Day Values,Snow Rain Ratio (unitless),Air Temperature Average (degC),Wind Speed Average (km/hr),Elevation (ft),Latitude,Longitude
0,1978-09-30,Baker Butte,308,NaN,NaN,NaN,NaN,NaN,0.0,NaN,7300,34.4566,-111.40643
1,1978-10-01,Baker Butte,308,NaN,NaN,NaN,NaN,NaN,0.0,NaN,7300,34.4566,-111.40643
2,1978-10-02,Baker Butte,308,NaN,NaN,NaN,NaN,NaN,0.0,NaN,7300,34.4566,-111.40643
3,1978-10-03,Baker Butte,308,NaN,NaN,NaN,NaN,NaN,0.0,NaN,7300,34.4566,-111.40643
4,1978-10-04,Baker Butte,308,NaN,NaN,NaN,NaN,NaN,0.0,NaN,7300,34.4566,-111.40643


In [15]:
# Standardizing column names for snow data.
column_names = {'Station_Name' : 'Station Name',
               'Station_Id' : 'Station ID',
               'Snow_Water_Equivalent__mm__Start_of_Day_Values' : 'SWE',
               'Snow_Depth__cm__Start_of_Day_Values' : 'Snow Depth',
               'Elevation__ft_' : 'Elevation',
               'Station Id' : 'Station ID',
               'Snow Water Equivalent (mm) Start of Day Values' : 'SWE',
               'Snow Depth (cm) Start of Day Values' : 'Snow Depth',
               'Snow Density (pct) Start of Day Values' : 'Snow Density',
               'Precipitation Accumulation (mm) Start of Day Values' : 'Precip Accumulation',
               'Snow Rain Ratio (unitless)' : 'Snow / Rain',
               'Air Temperature Average (degC)' : 'Average Air Temperature',
               'Wind Speed Average (km/hr)' : 'Average Wind Speed',
               'Elevation (ft)' : 'Elevation'}

for i in snow_states:
    raw_snow[i] = raw_snow[i].rename(columns=column_names)
    
# Standardizing column names for water data.
wa_column_names = {'Station_Name' : 'Station Name',
                   'Station_Id' : 'Station ID',
                   'Station Id' : 'Station ID',
                   'Reservoir Storage Volume (dam^3) Start of Day Values' : 'Water Storage',
                   'Reservoir_Storage_Volume__dam_3__Start_of_Day_Values' : 'Water Storage',
                   'Elevation (ft)' : 'Elevation',
                   'Elevation__ft_' : 'Elevation'}

for i in water_states:
    raw_water[i] = raw_water[i].rename(columns=wa_column_names)

In [26]:
# Combine the dataframes into one dataframe for snow data.
snow_data = pd.concat(raw_snow, axis=0)
water_data = pd.concat(raw_water, axis=0)

In [27]:
water_data

Date           Station Name  Station ID  Water Storage  \
AZ 0      1964-12-21   Cragin Dam Reservoir   9398300.0            0.0   
   1      1964-12-22   Cragin Dam Reservoir   9398300.0            NaN   
   2      1964-12-23   Cragin Dam Reservoir   9398300.0            NaN   
   3      1964-12-24   Cragin Dam Reservoir   9398300.0            NaN   
   4      1964-12-25   Cragin Dam Reservoir   9398300.0            NaN   
...              ...                    ...         ...            ...   
WY 59795  2022-03-29  Meeks Cabin Reservoir   9218400.0        15393.0   
   59796  2022-03-30  Meeks Cabin Reservoir   9218400.0        15470.0   
   59797  2022-03-31  Meeks Cabin Reservoir   9218400.0        15556.0   
   59798  2022-04-01  Meeks Cabin Reservoir   9218400.0        15642.0   
   59799  2022-04-02  Meeks Cabin Reservoir   9218400.0        15706.0   

          Elevation  Latitude  Longitude  OID_  
AZ 0         6620.0  34.55528 -111.18333   NaN  
   1         6620.0  34.55528 -111.18333   NaN  
   2         6620.0  34.55528 -111.18333   NaN  
   3         6620.0  34.55528 -111.18333   NaN  
   4         6620.0  34.55528 -111.18333   NaN  
...             ...       ...        ...   ...  
WY 59795     8673.0  41.02583 -110.58067   NaN  
   59796     8673.0  41.02583 -110.58067   NaN  
   59797     8673.0  41.02583 -110.58067   NaN  
   59798     8673.0  41.02583 -110.58067   NaN  
   59799     8673.0  41.02583 -110.58067   NaN  

[1010474 rows x 8 columns]

In [28]:
# Column Selection for master snow dataset
junk_columns = ['Station ID', 'Snow Density', 'Precip Accumulation', 'Snow / Rain', 'Average Air Temperature', 
                'Average Wind Speed', 'Elevation', 'Latitude', 'Longitude', 'OID_']

snow_data.drop(columns=junk_columns, inplace=True)

# Column Selection for master water dataset
water_data.drop(columns=['Station ID', 'Elevation', 'Latitude', 'Longitude', 'OID_'], inplace=True)

In [29]:
# Date time formatting
snow_data.index = pd.to_datetime(snow_data['Date'], infer_datetime_format=True)
snow_data.drop(columns='Date', inplace=True)

water_data.index = pd.to_datetime(water_data['Date'], infer_datetime_format=True)
water_data.drop(columns='Date', inplace=True)

In [30]:
snow_data

,Date,Station Name,SWE,Snow Depth
Date,,,,
1978-09-30,1978-09-30,Baker Butte,NaN,NaN
1978-10-01,1978-10-01,Baker Butte,NaN,NaN
1978-10-02,1978-10-02,Baker Butte,NaN,NaN
1978-10-03,1978-10-03,Baker Butte,NaN,NaN
1978-10-04,1978-10-04,Baker Butte,NaN,NaN
...,...,...,...,...
2022-03-31,3/31/2022 0:00:00,Whiskey Park,569.0,155.0
2022-04-01,4/1/2022 0:00:00,Whiskey Park,569.0,152.0
2022-04-02,4/2/2022 0:00:00,Whiskey Park,574.0,150.0
